In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
import sagemaker,boto3,os
from sagemaker import get_execution_role
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [12]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/defaulter-prediction-xgboost'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [13]:
bucket_name = "ml-case-study-data"
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [14]:
# Read the data from S3 bucket
role = get_execution_role()
conn = boto3.client('s3')
bucket = "ml-case-study-data"
dataFile = "dataset.csv"
response = conn.get_object(Bucket=bucket, Key=dataFile)
body = response['Body']
full_data = pd.read_csv(body,sep=";",index_col=0)

In [15]:
# Splitting the dataset for training and actual data based on the availability of the target variable
model_data = full_data.loc[(full_data['default'] == 0.0) | (full_data['default'] == 1.0)]
pred_data = full_data.loc[(full_data['default'] != 0.0) & (full_data['default'] != 1.0)]

In [16]:
model_data.head()

,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,account_worst_status_3_6m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
uuid,,,,,,,,,,,,,,,,,,,,,
63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0,...,1,1,1,1,0,0,0,178839,9.653333,1.0
0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [17]:
model_data.shape

(89976, 42)

In [18]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(62983, 42) (26993, 42)


In [19]:
colList = train_data.columns
colList

Index(['default', 'account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m',
       'account_incoming_debt_vs_paid_0_24m', 'account_status',
       'account_worst_status_0_3m', 'account_worst_status_12_24m',
       'account_worst_status_3_6m', 'account_worst_status_6_12m', 'age',
       'avg_payment_span_0_12m', 'avg_payment_span_0_3m', 'merchant_category',
       'merchant_group', 'has_paid', 'max_paid_inv_0_12m',
       'max_paid_inv_0_24m', 'name_in_email',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',
       

In [20]:
train_data['default'] = [0 if i == 0.0 else 1 for i in list(train_data['default'])]

In [21]:
preprocessor_pipeline = joblib.load("preprocessor_pipeline.pkl")

In [22]:
train_dep = train_data.drop(['default'], axis=1)
test_dep = test_data.drop(['default'], axis=1)
train_dep.shape

(62983, 41)

In [26]:
train_transformed_data = preprocessor_pipeline.transform(train_dep)
test_transformed_data = preprocessor_pipeline.transform(test_dep)
train_transformed_data = (pd.DataFrame(train_transformed_data.todense()))
test_transformed_data = (pd.DataFrame(test_transformed_data.todense()))

In [27]:
train_transformed_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,-0.041121,-0.235426,-0.104643,-0.624554,-0.444847,-0.011517,-0.011754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.041121,-0.235426,-0.104643,-0.445545,-0.444847,-0.011517,-0.011754,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.041121,-0.235426,-0.104643,-0.535049,-0.444847,-0.011517,-0.011754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.041121,-0.235426,-0.104643,-1.027322,-0.444847,-0.011517,-0.011754,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.041121,-0.235426,-0.104643,-0.726844,-0.444847,-0.011517,-0.011754,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
pd.concat([train_data['default'], train_transformed_data], axis=1).to_csv('train.csv', index=False, header=False)
#pd.concat([test_data['default'], test_transformed_data, axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
#boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
#s3_input_validate = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [30]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [31]:
xgb.fit({'train': s3_input_train})

2022-01-06 15:53:55 Starting - Starting the training job...
2022-01-06 15:54:04 Starting - Launching requested ML instancesProfilerReport-1641484434: InProgress
......
2022-01-06 15:55:25 Starting - Preparing the instances for training............
2022-01-06 15:57:26 Downloading - Downloading input data...
2022-01-06 15:57:56 Training - Training image download completed. Training in progress..Arguments: train
[2022-01-06:15:57:57:INFO] Running standalone xgboost training.
[2022-01-06:15:57:57:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-01-06:15:57:57:INFO] File size need to be processed in the node: 49.73mb. Available memory size in the node: 8374.11mb
[2022-01-06:15:57:57:INFO] Determined delimiter of CSV input is ','
[15:57:57] S3DistributionType set as FullyReplicated
[15:57:58] 125966x144 matrix with 18076121 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[15:57:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 ext

In [32]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-----------!

In [34]:
xgb_predictor.endpoint_name

'xgboost-2022-01-06-16-00-03-558'

In [35]:
#Evaluate the model
def predict(data,rows=1000):
    split_array = np.array_split(data,int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions,xgb_predictor.predict(array)])
    return np.fromstring(predictions[1:],sep=',')

In [39]:
pd.concat([test_data['default'], test_transformed_data], axis=1).to_csv('validation.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [41]:
payload = pd.read_csv('validation.csv', header=None)
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(EndpointName=xgb_predictor.endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value:        0         1         2         3         4             5         6    \
0      NaN -0.041121 -0.235426 -0.104643  0.000000  5.383818e-17  0.000000   
1      NaN -0.041121 -0.235426 -0.104643 -0.266537 -4.448467e-01 -0.011517   
2      NaN -0.041121  1.110174 -0.104643  3.425507 -4.448467e-01 -0.011517   
3      NaN -0.041121  0.785374 -0.104643 -0.706275 -4.448467e-01 -0.011517   
4      NaN -0.041121 -0.235426 -0.104643  0.000000  5.383818e-17  0.000000   
...    ...       ...       ...       ...       ...           ...       ...   
53981  0.0       NaN       NaN       NaN       NaN           NaN       NaN   
53982  0.0       NaN       NaN       NaN       NaN           NaN       NaN   
53983  0.0       NaN       NaN       NaN       NaN           NaN       NaN   
53984  0.0       NaN       NaN       NaN       NaN           NaN       NaN   
53985  0.0       NaN       NaN       NaN       NaN 

In [36]:
import matplotlib as plt
predictions = predict(test_transformed_data)
plt.hist(predictions)
plt.show()

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value:           0         1         2         3             4         5         6    \
0   -0.041121 -0.235426 -0.104643  0.000000  5.383818e-17  0.000000  0.000000   
1   -0.041121 -0.235426 -0.104643 -0.266537 -4.448467e-01 -0.011517 -0.011754   
2   -0.041121  1.110174 -0.104643  3.425507 -4.448467e-01 -0.011517 -0.011754   
3   -0.041121  0.785374 -0.104643 -0.706275 -4.448467e-01 -0.011517 -0.011754   
4   -0.041121 -0.235426 -0.104643  0.000000  5.383818e-17  0.000000  0.000000   
..        ...       ...       ...       ...           ...       ...       ...   
995  0.000000  0.000000  0.000000 -0.714058 -4.448467e-01 -0.011517 -0.011754   
996 -0.041121 -0.235426 -0.104643 -0.266537  3.434603e+00 -0.011517 -0.011754   
997 -0.041121 -0.235426 -0.104643  0.078693 -4.448467e-01 -0.011517 -0.011754   
998 -0.041121 -0.235426 -0.104643  0.172696 -1.104114e-01 -0.011517 -0.011754   
999 -0.041121 -0.2